In [43]:
import math

# Constants
span = 20  # span of the bridge in meters
clear_width_roadway = 7.5  # clear width of roadway in meters
footpath_width = 1.0  # width of footpath on either side in meters
spacing_main_girders = 2.0  # spacing of main girders in meters
concrete_grade = 30  # M30 concrete
steel_grade = 415  # Fe-415 grade tor steel
yield_stress_steel = 236  # yield stress of rolled steel sections in N/mm^2

# Deck Slab Design
def deck_slab_design():
    # Panel dimensions
    panel_width = spacing_main_girders  # meters
    panel_length = 4.5  # meters

    # Dead loads
    slab_thickness = 0.3  # meters
    unit_weight_concrete = 24  # kN/m^3
    unit_weight_wearing_course = 22  # kN/m^3
    thickness_wearing_course = 0.08  # meters

    dead_weight_slab = slab_thickness * unit_weight_concrete  # kN/m^2
    dead_weight_wc = thickness_wearing_course * unit_weight_wearing_course  # kN/m^2
    total_dead_load = dead_weight_slab + dead_weight_wc  # kN/m^2

    # Live load moments
    W = 350  # Total load in kN
    m1 = 0.085  # Pigeaud coefficient for short span
    m2 = 0.017  # Pigeaud coefficient for long span
    short_span_moment = W * (m1 + 0.15 * m2)  # kNm
    long_span_moment = W * (m2 + 0.015 * m1)  # kNm

    # Impact and continuity factors
    impact_factor = 1.25
    continuity_factor = 0.8

    short_span_moment *= impact_factor * continuity_factor
    long_span_moment *= impact_factor * continuity_factor

    # Dead load moments
    total_panel_load = total_dead_load * panel_width * panel_length  # kN
    dead_load_short_span = total_panel_load * (0.047 + 0.15 * 0.006)  # kNm
    dead_load_long_span = total_panel_load * (0.006 + 0.15 * 0.047)  # kNm

    dead_load_short_span *= continuity_factor
    dead_load_long_span *= continuity_factor

    # Ultimate design moments
    ultimate_short_span_moment = 1.35 * dead_load_short_span + 1.5 * short_span_moment  # kNm
    ultimate_long_span_moment = 1.35 * dead_load_long_span + 1.5 * long_span_moment  # kNm

    # Reinforcement design
    effective_depth = 250  # mm
    slab_width = 1000  # mm (considering 1 meter width for calculations)
    
    # Short span reinforcement
    mu = ultimate_short_span_moment * 1e6  # convert kNm to Nmm
    pt = 0.138 * concrete_grade
    d = effective_depth
    p = mu / (pt * slab_width * d**2)
    area_short_span = p * slab_width * d / 100  # mm^2

    # Long span reinforcement
    mu = ultimate_long_span_moment * 1e6  # convert kNm to Nmm
    pt = 0.138 * concrete_grade
    p = mu / (pt * slab_width * d**2)
    area_long_span = p * slab_width * d / 100  # mm^2

    # Bar spacing
    bar_dia_short_span = 12  # mm
    bar_dia_long_span = 10  # mm
    spacing_short_span = (math.pi * (bar_dia_short_span / 2)**2) / (area_short_span / 1000) * 1000  # mm
    spacing_long_span = (math.pi * (bar_dia_long_span / 2)**2) / (area_long_span / 1000) * 1000  # mm

    return total_dead_load, ultimate_short_span_moment, ultimate_long_span_moment, area_short_span, area_long_span, spacing_short_span, spacing_long_span

# Girder Design
def girder_design():
    span_length = span  # meters
    self_weight_girder = 4  # kN/m (assumed)
    total_dead_load_girder = 18 + self_weight_girder  # kN/m

    # Maximum dead load moment
    max_dead_load_moment = (total_dead_load_girder * span_length**2) / 8  # kNm

    # Live load moment
    live_load_moment = (350 * span_length) / 2 - (35 * 0.9) / 2  # kNm
    impact_factor = 1.10
    live_load_moment *= impact_factor

    # Design moment
    design_moment = max_dead_load_moment + live_load_moment  # kNm

    # Shear forces
    dead_load_shear = (total_dead_load_girder * span_length) / 2  # kN
    live_load_shear = (350 * span_length * 0.9) / 2  # kN
    total_shear = dead_load_shear + live_load_shear  # kN

    # Girder proportions
    depth_of_girder = 2000  # mm
    width_of_flange = 500  # mm (assumed)
    flange_thickness = 30  # mm (assumed)
    web_thickness = 10  # mm

    # Check for stresses
    I_girder = (web_thickness * depth_of_girder**3) / 12 + 2 * (flange_thickness * width_of_flange * (depth_of_girder - flange_thickness)**2) / 12  # mm^4
    y = depth_of_girder / 2  # mm

    bending_stress = (design_moment * 1e6 * y) / I_girder  # N/mm^2
    shear_stress = total_shear * 1e3 / (depth_of_girder * web_thickness)  # N/mm^2

    return depth_of_girder, width_of_flange, bending_stress, shear_stress

# Shear Connectors Design
def shear_connectors_design():
    shear_force = 204  # kN
    connector_diameter = 20  # mm
    fck = 30  # N/mm^2
    Q = 196 * connector_diameter**2 * math.sqrt(fck)  # N
    connectors_per_row = 2
    pitch = (connectors_per_row * Q) / (2 * shear_force * 1e3) * 1000  # mm

    return connectors_per_row, pitch

# Stiffeners Design
def stiffeners_design():
    web_height = 1000  # mm
    web_thickness = 10  # mm
    stiffener_spacing_min = 0.33 * web_height  # mm
    stiffener_spacing_max = 1.5 * web_height  # mm

    spacing_of_stiffeners = (stiffener_spacing_min + stiffener_spacing_max) / 2  # mm
    stiffener_area = web_thickness * 80  # mm^2 (using a 10 mm x 80 mm plate)

    return spacing_of_stiffeners, stiffener_area

# Run calculations
deck_load, deck_short_moment, deck_long_moment, deck_short_reinf, deck_long_reinf, deck_short_spacing, deck_long_spacing = deck_slab_design()
girder_depth, girder_flange_width, girder_bend_stress, girder_shear_stress = girder_design()
connectors_count, connectors_pitch = shear_connectors_design()
stiffener_spacing, stiffener_area = stiffeners_design()

# Print Results
print(f"Deck Slab Total Dead Load: {deck_load:.2f} kN/m^2")
print(f"Deck Slab Ultimate Short Span Moment: {deck_short_moment:.2f} kNm")
print(f"Deck Slab Ultimate Long Span Moment: {deck_long_moment:.2f} kNm")
print(f"Deck Slab Short Span Reinforcement Area: {deck_short_reinf:.2f} mm^2")
print(f"Deck Slab Long Span Reinforcement Area: {deck_long_reinf:.2f} mm^2")
print(f"Deck Slab Short Span Bar Spacing (12mm dia): {deck_short_spacing:.2f} mm")
print(f"Deck Slab Long Span Bar Spacing (10mm dia): {deck_long_spacing:.2f} mm")

print(f"Girder Depth: {girder_depth} mm")
print(f"Girder Width of Flange: {girder_flange_width} mm")
print(f"Girder Bending Stress: {girder_bend_stress:.2f} N/mm^2")
print(f"Girder Shear Stress: {girder_shear_stress:.2f} N/mm^2")

print(f"Shear Connectors per Row: {connectors_count}")
print(f"Shear Connectors Pitch: {connectors_pitch:.2f} mm")

print(f"Spacing of Stiffeners: {stiffener_spacing:.2f} mm")
print(f"Area of Stiffeners: {stiffener_area:.2f} mm^2")


Deck Slab Total Dead Load: 8.96 kN/m^2
Deck Slab Ultimate Short Span Moment: 50.14 kNm
Deck Slab Ultimate Long Span Moment: 10.73 kNm
Deck Slab Short Span Reinforcement Area: 484.40 mm^2
Deck Slab Long Span Reinforcement Area: 103.68 mm^2
Deck Slab Short Span Bar Spacing (12mm dia): 233479.14 mm
Deck Slab Long Span Bar Spacing (10mm dia): 757518.89 mm
Girder Depth: 2000 mm
Girder Width of Flange: 500 mm
Girder Bending Stress: 301.34 N/mm^2
Girder Shear Stress: 168.50 N/mm^2
Shear Connectors per Row: 2
Shear Connectors Pitch: 2104.97 mm
Spacing of Stiffeners: 915.00 mm
Area of Stiffeners: 800.00 mm^2


In [32]:
import math

# Constants and inputs
vertical_reaction = 1200  # kN
horizontal_reaction = 120  # kN
permissible_compressive_stress_concrete = 4  # N/mm^2
permissible_bending_stress_steel = 160  # N/mm^2
permissible_bearing_stress_steel = 185  # N/mm^2
permissible_shear_stress_steel = 105  # N/mm^2
modulus_of_elasticity = 200  # kN/mm^2
ultimate_tensile_strength = 250  # N/mm^2

# Bed Plate Design
area_bed_plate = (vertical_reaction * 10**3) / 4 - 25 * 10**4  # mm^2
provided_size_bed_plate = (400, 650, 40)  # mm
area_provided_bed_plate = provided_size_bed_plate[0] * provided_size_bed_plate[1]  # mm^2

# Rocker Diameter
design_load_per_unit_length = (vertical_reaction * 10**3) / 650  # N/mm
radius_squared = (design_load_per_unit_length * 200 * 10**3) / (170 * 250**3)
radius = math.sqrt(radius_squared)  # mm
# Adopt a radius of 200 mm
radius = 200

# Rocker Pin Design
horizontal_shear_force_per_pin = horizontal_reaction / 2  # kN
diameter_squared = (horizontal_shear_force_per_pin * 10**3) / (permissible_shear_stress_steel * math.pi)
diameter = math.sqrt(diameter_squared * 4 / math.pi)  # mm
# Adopt tapering pin with top diameter of 25 mm and bottom diameter of 30 mm
diameter_top = 25  # mm
diameter_bottom = 30  # mm
height_pin = 55  # mm

# Base Plate Thickness
max_bending_moment = (500 * 10**3 * 100)  # N-mm
section_modulus_required = max_bending_moment / permissible_bending_stress_steel  # mm^3
width_base_plate = 650  # mm
thickness_squared = (section_modulus_required * 6) / width_base_plate
thickness = math.sqrt(thickness_squared)  # mm
# Provide a thickness of 72 mm
thickness = 72

# Bearing Stress Check
contact_area = 0.5 * (provided_size_bed_plate[0] * provided_size_bed_plate[1])  # mm^2
bearing_stress = (vertical_reaction * 10**3) / contact_area  # N/mm^2

# Results
print("Bed Plate Design:")
print(f"Area required: {area_bed_plate:.2f} mm^2")
print(f"Provided size: {provided_size_bed_plate[0]} mm x {provided_size_bed_plate[1]} mm")
print(f"Area provided: {area_provided_bed_plate:.2f} mm^2")

print("\nRocker Diameter Design:")
print(f"Design load per unit length: {design_load_per_unit_length:.2f} N/mm")
print(f"Calculated radius squared: {radius_squared:.2f} mm^2")
print(f"Adopted radius: {radius} mm")

print("\nRocker Pin Design:")
print(f"Horizontal shear force per pin: {horizontal_shear_force_per_pin:.2f} kN")
print(f"Calculated diameter: {diameter:.2f} mm")
print(f"Adopted top diameter: {diameter_top} mm")
print(f"Adopted bottom diameter: {diameter_bottom} mm")
print(f"Height of pin: {height_pin} mm")

print("\nBase Plate Thickness Design:")
print(f"Maximum bending moment: {max_bending_moment:.2f} N-mm")
print(f"Required section modulus: {section_modulus_required:.2f} mm^3")
print(f"Adopted thickness: {thickness} mm")

print("\nBearing Stress Check:")
print(f"Contact area: {contact_area:.2f} mm^2")
print(f"Bearing stress: {bearing_stress:.2f} N/mm^2")
if bearing_stress < permissible_bearing_stress_steel:
    print("Bearing stress is within safe permissible limits.")
else:
    print("Bearing stress exceeds permissible limits.")



Bed Plate Design:
Area required: 50000.00 mm^2
Provided size: 400 mm x 650 mm
Area provided: 260000.00 mm^2

Rocker Diameter Design:
Design load per unit length: 1846.15 N/mm
Calculated radius squared: 0.14 mm^2
Adopted radius: 200 mm

Rocker Pin Design:
Horizontal shear force per pin: 60.00 kN
Calculated diameter: 15.22 mm
Adopted top diameter: 25 mm
Adopted bottom diameter: 30 mm
Height of pin: 55 mm

Base Plate Thickness Design:
Maximum bending moment: 50000000.00 N-mm
Required section modulus: 312500.00 mm^3
Adopted thickness: 72 mm

Bearing Stress Check:
Contact area: 130000.00 mm^2
Bearing stress: 9.23 N/mm^2
Bearing stress is within safe permissible limits.


In [19]:
import math

# Constants and inputs
width_pier_hfl = 19  # m
submerged_volume_pier = 187.4  # m^3
density_concrete = 24  # kN/m^3
height_pier = 8.5  # m
width_pier_base = 8  # m, assume based on previous calculations
depth_pier_base = 3  # m, assume based on previous calculations

# Load values
dead_load_superstructure = 2 * 2000  # kN
self_weight_pier = 5100  # kN
total_direct_load = dead_load_superstructure + self_weight_pier  # kN
buoyancy_reduction = submerged_volume_pier * density_concrete  # kN
live_load_reaction = 1000  # kN
eccentricity = 0.5  # m
braking_force = 140  # kN
wind_pressure = 2.4  # kN/m^2
wind_area = width_pier_base * height_pier  # m^2
wind_load = wind_area * wind_pressure  # kN

# 1. Stress due to dead load and self-weight of pier
total_load = total_direct_load  # kN
area_base_pier = width_pier_base * depth_pier_base  # m^2
stress_dead_load_self_weight = total_load * 1e3 / area_base_pier  # N/m^2 to kN/m^2

# 2. Effect of buoyancy
effective_weight = total_load - buoyancy_reduction  # kN
stress_buoyancy = effective_weight * 1e3 / area_base_pier  # N/m^2 to kN/m^2

# 3. Due to eccentricity of live load
moment_due_to_live_load = live_load_reaction * eccentricity  # kNm
section_modulus = (width_pier_base * depth_pier_base**2) / 6  # m^3, for a rectangular section
stress_eccentricity = moment_due_to_live_load * 1e6 / section_modulus  # N/m^2 to kN/m^2

# 4. Due to longitudinal braking forces
moment_due_to_braking = braking_force * 10  # kNm, assume a distance of 10 m
stress_braking = moment_due_to_braking * 1e6 / section_modulus  # N/m^2 to kN/m^2

# 5. Due to wind pressure
moment_due_to_wind = wind_load * (height_pier / 2)  # kNm, assuming wind acts at mid-height
stress_wind = moment_due_to_wind * 1e6 / section_modulus  # N/m^2 to kN/m^2

# Output
print(f"Stress due to dead load and self-weight of pier: {stress_dead_load_self_weight:.2f} kN/m²")
print(f"Stress due to buoyancy: {stress_buoyancy:.2f} kN/m²")
print(f"Stress due to eccentricity of live load: {stress_eccentricity:.2f} kN/m²")
print(f"Stress due to longitudinal braking forces: {stress_braking:.2f} kN/m²")
print(f"Stress due to wind pressure: {stress_wind:.2f} kN/m²")

# Maximum and minimum stresses
max_stress = max(stress_dead_load_self_weight, stress_buoyancy + stress_eccentricity, stress_braking, stress_wind)
min_stress = min(stress_dead_load_self_weight, stress_buoyancy - stress_eccentricity, stress_braking, stress_wind)

print(f"Maximum stress developed at the base of the pier: {max_stress:.2f} kN/m²")
print(f"Minimum stress developed at the base of the pier: {min_stress:.2f} kN/m²")


Stress due to dead load and self-weight of pier: 379166.67 kN/m²
Stress due to buoyancy: 191766.67 kN/m²
Stress due to eccentricity of live load: 41666666.67 kN/m²
Stress due to longitudinal braking forces: 116666666.67 kN/m²
Stress due to wind pressure: 57800000.00 kN/m²
Maximum stress developed at the base of the pier: 116666666.67 kN/m²
Minimum stress developed at the base of the pier: -41474900.00 kN/m²


In [33]:
# Constants
density_stone_masonry = 25  # kN/m^3
width_rectangular = 1  # meters
height_rectangular = 3  # meters
width_triangular = 0.5  # meters
height_triangular = 1  # meters
live_load = 20  # kN
earth_load = 15  # kN

# Weights
Wx = width_rectangular * height_rectangular * density_stone_masonry
W2 = 0.5 * width_triangular * height_triangular * density_stone_masonry

# Total vertical forces
W_total = Wx + W2 + live_load + earth_load

# Moments
M_toe_A = (95 * 0.5) + (37.5 * 1.33) + (15 * 1.67) - (20 * 1)
e = M_toe_A / W_total

# Stress calculations
b = 2  # meters (base width)
L = 1  # meters (base length)
sigma_A = (W_total / (b * L)) + (6 * e / b)
sigma_B = (W_total / (b * L)) - (6 * e / b)

# Stability Check
friction_coefficient = 0.5
horizontal_force = 20  # kN

# Factional Resistance
frictional_resistance = friction_coefficient * W_total

# Factor of Safety
factor_of_safety = frictional_resistance / horizontal_force

# Print results
print(f"Total Vertical Forces: {W_total} kN")
print(f"Moment about Toe A: {M_toe_A} kNm")
print(f"Eccentricity: {e} m")
print(f"Maximum Stress at Base: {sigma_A} kN/m^2")
print(f"Minimum Stress at Base: {sigma_B} kN/m^2")
print(f"Factor of Safety against Sliding: {factor_of_safety}")

# Check against permissible limits
safe_bearing_capacity = 150  # kN/m^2
print(f"Maximum Stress is {'within' if sigma_A < safe_bearing_capacity else 'exceeding'} safe limits.")
print(f"Factor of Safety is {'adequate' if factor_of_safety > 2.00 else 'inadequate'}.")


Total Vertical Forces: 116.25 kN
Moment about Toe A: 102.425 kNm
Eccentricity: 0.8810752688172043 m
Maximum Stress at Base: 60.76822580645161 kN/m^2
Minimum Stress at Base: 55.48177419354839 kN/m^2
Factor of Safety against Sliding: 2.90625
Maximum Stress is within safe limits.
Factor of Safety is adequate.


In [35]:
import math

# Given data
d_i = 2.5  # Internal diameter of well in meters
L = 25  # Depth of well below bed level in meters
K = 0.033  # Constant for clayee soil
M20_density = 2500  # Density of M20 concrete in kg/m^3 (assuming)
Fe415_density = 7850  # Density of Fe415 steel in kg/m^3 (assuming)
sigma_safe = 150  # Safe bearing capacity of soil in kN/m^2
W_total = 200  # Total vertical load from pier and abutment in kN

# 1. Thickness of Steining
h = K * (d_i + 2 * h) * math.sqrt(L)
h_approx = 0.558  # Calculated from previous steps
h_final = 600 / 1000  # Adopting 600 mm steining thickness

# 2. Reinforcement in Steining
# Area of cross-section
A_gross = math.pi * (((d_i + 2 * h_final) / 2) ** 2 - (d_i / 2) ** 2)

# Minimum longitudinal reinforcement (0.2% of gross cross-sectional area)
A_min_longitudinal = 0.002 * A_gross * 1000000  # in mm^2
A_single_bar = math.pi * (16 / 2) ** 2  # Area of a single 16 mm diameter bar
num_bars_per_face = A_min_longitudinal / A_single_bar

# Hoop reinforcement
volume_per_unit_length = math.pi * (((d_i + 2 * h_final) ** 2 - d_i ** 2) / 4)
hoop_reinforcement_volume = 0.0004 * volume_per_unit_length
weight_per_meter = hoop_reinforcement_volume * Fe415_density
circumference_avg = math.pi * (d_i + h_final)
weight_one_hoop = circumference_avg * math.pi * (10 / 1000) ** 2 * Fe415_density
num_hoops_per_meter = weight_per_meter / weight_one_hoop
spacing_hoops = 1 / num_hoops_per_meter

# 3. Well Curb
volume_curb = math.pi * ((d_i + 2 * h_final) ** 2 - d_i ** 2) / 4 * 1
steel_required_curb = 72 * volume_curb
weight_hoop_20mm = circumference_avg * 20 / 1000 * Fe415_density
weight_hoop_16mm = circumference_avg * 16 / 1000 * Fe415_density
weight_tie_8mm = 3 * math.pi * (8 / 2 / 1000) ** 2 * Fe415_density
num_ties = math.ceil(math.pi * (d_i + h_final) / 0.3)
weight_ties = num_ties * weight_tie_8mm
weight_8_hoops_20mm = 8 * weight_hoop_20mm
weight_6_hoops_16mm = 6 * weight_hoop_16mm
total_steel_curb = weight_8_hoops_20mm + weight_ties + weight_6_hoops_16mm

# 4. Check Bearing Capacity
base_area = math.pi * ((d_i + 2 * h_final) / 2) ** 2
actual_bearing_pressure = W_total / base_area

# 5. Check Stability against Overturning and Sliding
H = 50  # Horizontal force assumption in kN
M_r = sigma_safe * base_area * (L / 2)
M_o = H * L
F_r = 0.5 * W_total  # Assuming friction coefficient of 0.5

# Output results
print(f"Thickness of Steining (h): {h_final * 1000:.2f} mm")
print(f"Minimum Longitudinal Reinforcement per face: {num_bars_per_face:.0f} bars of 16 mm diameter")
print(f"Hoop Reinforcement: Use 10 mm diameter hoops at {spacing_hoops * 1000:.0f} mm centers")
print(f"Total Steel in Well Curb: {total_steel_curb:.2f} kg")
print(f"Actual Bearing Pressure: {actual_bearing_pressure:.2f} kN/m^2 (Safe Bearing Capacity: {sigma_safe} kN/m^2)")
print(f"Overturning Moment (M_r): {M_r:.2f} kNm")
print(f"Overturning Moment (M_o): {M_o:.2f} kNm")
print(f"Resisting Force against Sliding (F_r): {F_r:.2f} kN")
print(f"Horizontal Force (H): {H:.2f} kN")

# Stability checks
if actual_bearing_pressure < sigma_safe:
    print("Bearing capacity is safe.")
else:
    print("Bearing capacity is NOT safe.")

if M_r > M_o:
    print("Safe against overturning.")
else:
    print("NOT safe against overturning.")

if F_r > H:
    print("Safe against sliding.")
else:
    print("NOT safe against sliding.")


Thickness of Steining (h): 600.00 mm
Minimum Longitudinal Reinforcement per face: 58 bars of 16 mm diameter
Hoop Reinforcement: Use 10 mm diameter hoops at 1309 mm centers
Total Steel in Well Curb: 19610.43 kg
Actual Bearing Pressure: 18.60 kN/m^2 (Safe Bearing Capacity: 150 kN/m^2)
Overturning Moment (M_r): 20160.19 kNm
Overturning Moment (M_o): 1250.00 kNm
Resisting Force against Sliding (F_r): 100.00 kN
Horizontal Force (H): 50.00 kN
Bearing capacity is safe.
Safe against overturning.
Safe against sliding.
